In [1]:
%pip install --upgrade openai
%pip install --upgrade tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from openai import OpenAI
from tqdm import tqdm

from src.helper_func.file_util import *
from src.helper_func.graph_util import *

In [3]:
# If True: do not call the API, just print the prompts
only_print_prompt = False

In [4]:
config = read_ini_file('../config.ini')
project_name = config['project']['name']
project_desc = config['project']['desc']
ifile = config['project']['ifile']
(project_name,project_desc,ifile)

('bitcoin-wallet-v6.31',
 'Bitcoin Wallet, a standalone Bitcoin payment app for your Android device!',
 'D:\\KULIAH\\SEMESTER 8\\Tugas Akhir II\\HnV-software-knowledge-analysis\\test\\case-studies-src\\extracted-knowledge-graph\\bitcoin-wallet-v6.31.json')

In [5]:
graph = read_json_file(ifile)
nodes,edges = transform_graph(graph)

In [6]:
nodes

{'de.schildbach.wallet.ui.WalletBalanceFragment.onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater).inflater': {'id': 'de.schildbach.wallet.ui.WalletBalanceFragment.onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater).inflater',
  'properties': {'simpleName': 'inflater',
   'qualifiedName': 'de.schildbach.wallet.ui.WalletBalanceFragment.onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater).inflater',
   'kind': 'parameter',
   'metaSrc': 'source code'},
  'labels': ['Variable']},
 'de.schildbach.wallet.ui.TransactionsAdapter.de.schildbach.wallet.ui.TransactionsAdapter(android.content.Context,int,de.schildbach.wallet.ui.TransactionsAdapter$OnClickListener).context': {'id': 'de.schildbach.wallet.ui.TransactionsAdapter.de.schildbach.wallet.ui.TransactionsAdapter(android.content.Context,int,de.schildbach.wallet.ui.TransactionsAdapter$OnClickListener).context',
  'properties': {'simpleName': 'context',
   'qualifiedName': 'de.schildbach.wallet.ui.Transact

In [7]:
cliet_args = dict()

if 'apikey' in config['openai']:
  cliet_args['api_key'] = config['openai']['apikey']
if 'apibase' in config['openai']:
  cliet_args['base_url'] = config['openai']['apibase']
if 'model' in config['openai']:
  model = config['openai']['model']
else:
  model = "gpt-3.5-turbo"

(list(cliet_args.keys()), model)

(['api_key', 'base_url'], 'gpt-3.5-turbo-0125')

In [8]:
client = OpenAI(**cliet_args)
client.base_url

URL('https://api.openai.com/v1/')

In [9]:
# test the LLM server---create a completion
completion = client.chat.completions.create(
  model=model,
  messages=[{"role":"user","content":"What is your name?"}],
  temperature=0
)
# print the completion
print(completion.choices[0].message.content)

I am an AI digital assistant and do not have a personal name. You can simply refer to me as Assistant. How can I assist you today?


In [10]:
methods = sorted([(pkg_id,cls_id,met_id) for pkg_id,cls_id,met_id in find_paths(edges['contains'], edges['hasScript']) if nodes[met_id]['properties']['visibility']=='public'])
len(methods)

912

In [11]:
classes = sorted({(pkg,clz) for pkg,clz,_ in methods})
len(classes)

218

In [12]:
packages = sorted({pkg for pkg,_ in classes})
len(packages),packages

(13,
 ['de.schildbach.wallet',
  'de.schildbach.wallet.data',
  'de.schildbach.wallet.integration.android',
  'de.schildbach.wallet.integration.sample',
  'de.schildbach.wallet.offline',
  'de.schildbach.wallet.service',
  'de.schildbach.wallet.ui',
  'de.schildbach.wallet.ui.backup',
  'de.schildbach.wallet.ui.monitor',
  'de.schildbach.wallet.ui.preference',
  'de.schildbach.wallet.ui.scan',
  'de.schildbach.wallet.ui.send',
  'de.schildbach.wallet.util'])

In [13]:
results = {pkg_id:{
	'qualifiedName': nodes[pkg_id]['properties']['qualifiedName'],
	'classes': {cls_id: {
		'qualifiedName': nodes[cls_id]['properties']['qualifiedName'],
		'methods': {met_id: {
			'qualifiedName': nodes[met_id]['properties']['qualifiedName']
		} for _,c,met_id in methods if c == cls_id}
	} for p,cls_id in classes if p == pkg_id}
} for pkg_id in packages}

results

{'de.schildbach.wallet': {'qualifiedName': 'de.schildbach.wallet',
  'classes': {'de.schildbach.wallet.Configuration': {'qualifiedName': 'de.schildbach.wallet.Configuration',
    'methods': {'de.schildbach.wallet.Configuration.armBackupReminder()': {'qualifiedName': 'de.schildbach.wallet.Configuration.armBackupReminder()'},
     'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)': {'qualifiedName': 'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)'},
     'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)': {'qualifiedName': 'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)'},
     'de.schildbach.wallet.Configuration.disarmBackupReminder()': {'qualifiedName': 'de.schildbach.wallet.Configuration.disarmBackupReminder()'},
     'de.schildbach.wallet.Configuration.getBestChainH

In [14]:
prompt1_template = '''**Layered Software Architecture and Responsibilities:**

1. **Presentation Layer**: Manages the user interface, defines UI elements and behavior, displays information, responds to user input, and updates views.
  
2. **Service Layer**: Controls the application flow, orchestrates domain operations, connects UI events with domain logic, and synchronizes domain changes with the UI.

3. **Domain Layer**: Handles business logic, represents domain data and behavior, and performs necessary computations for domain operations.

4. **Data Source Layer**: Interacts with databases, filesystems, hardware, messaging systems, or other data sources, performs CRUD operations, handles data conversion, and ensures data integrity.

**Project Context**: {project_desc}

Consider the following Java method from the class {class_name} in the project:

```java
{method_src}
```

**Task**:

1. **Summarize the Method**: Briefly describe the main responsibility of this method (1-2 sentences).
2. **Layer Evaluation**: Evaluate by comparing the method’s responsibility with each layer's role described above. Then, identify which layer(s) the method best fits into based on the evaluation and the project context, and explain your choice.
3. **Justification**:
    - **Multiple Layers**: If the method could belong to more than one layer, evaluate each possibility. Argue for the best fit and provide clear reasoning.
    - **Single Layer**: If the method clearly fits one layer, justify your choice with specific examples and reasoning.
    - **No Layer**: If the method doesn’t fit any layer, check if it could partially fit any layer. If not, classify it as "Cross-Cutting" and explain your reasoning.'''

prompt2 = "In conclusion, state the single layer that you think fits this method the most. Just answer with the name of the layer and nothing else."

In [15]:
import time

timestr = time.strftime("%Y%m%d-%H%M%S")
timestr

'20240613-002903'

In [16]:
layer_names = [
	"Presentation Layer",
 	"Service Layer",
	"Domain Layer",
 	"Data Source Layer",
  	# "Cross-Cutting"
]

In [17]:
with open(f'{project_name}-layering-{timestr}.log', 'a') as file:

    kind = 'class'
    current_pkg = None
    current_cls = None
    
    for pkg_id,cls_id,met_id in tqdm(methods, desc='Processing methods'):
            
        if current_pkg != pkg_id:
            if current_pkg:
                file.write('\n\n===============================================\n\n')
                
            last_pkg = current_pkg
            current_pkg = pkg_id
            file.write('# ' + current_pkg + "\n")
        
        if current_cls != cls_id:
            file.flush()
            last_cls = current_cls
            current_cls = cls_id
            file.write('\t* ' + current_cls + "\n")

        if not 'layer' in results[pkg_id]['classes'][cls_id]['methods'][met_id] \
                or not results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer'] in layer_names:

            file.write('\t\t- ' + met_id + "\n")

            package = nodes[pkg_id]
            clasz = nodes[cls_id]
            method = nodes[met_id]

            method_name = method['properties']['simpleName']
            
            prompt1 = prompt1_template.format(
                project_desc=project_desc,
                method_src=method['properties']["sourceText"],
                class_name=clasz['properties']['qualifiedName']
            )
            if only_print_prompt:
                file.write('\t\t\t' + prompt1)
                file.write("\n\n")
            else:
                response = None
                try:
                    response = client.chat.completions.create(
                        model=model,
                        messages=[{
                            "role": "user",
                            "content": prompt1
                        }],
                        temperature=0)
                    ast_message = response.choices[0].message
                    
                    file.write('\t\t\t' + "[USER]\n\n")
                    file.write('\t\t\t' + prompt1)
                    file.write("\n\n")
                    file.write('\t\t\t' + "[LLM]\n\n")
                    file.write('\t\t\t' + ast_message.content)
                    file.write("\n\n")

                    response = client.chat.completions.create(
                        model=model,
                        messages=[{
                            "role": "user",
                            "content": prompt1
                        }, 
                        ast_message,
                        {
                            "role": "user",
                            "content": prompt2
                        }],
                        temperature=0)
                    answer = response.choices[0].message.content
                    
                    file.write('\t\t\t' + "[USER]\n\n")
                    file.write('\t\t\t' + prompt2)
                    file.write("\n\n")
                    file.write('\t\t\t' + "[LLM]\n\n")
                    file.write('\t\t\t' + answer)
                    file.write("\n\n")

                    results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer'] = answer
                except:
                    answer = None
                    file.write('\t\t\t' + (str(response) if response else "no response"))
                    file.write("\n\n")
                    results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer'] = "undefined"
                    
            file.write("\n\n")

    file.write("ALL RESULTS:\n\n")
    file.write(prettify_json(results))

Processing methods: 100%|██████████| 912/912 [1:37:25<00:00,  6.41s/it]


In [18]:
if not only_print_prompt:
  write_to_json_file(results, f"../../test/2-architecture-layering-output/{project_name}-layers-{model}-{timestr}.json")

In [19]:
results = read_json_file(f"../../test/2-architecture-layering-output/{project_name}-layers-{model}-{timestr}.json")
results

{'de.schildbach.wallet': {'qualifiedName': 'de.schildbach.wallet',
  'classes': {'de.schildbach.wallet.Configuration': {'qualifiedName': 'de.schildbach.wallet.Configuration',
    'methods': {'de.schildbach.wallet.Configuration.armBackupReminder()': {'qualifiedName': 'de.schildbach.wallet.Configuration.armBackupReminder()',
      'layer': 'Data Source Layer'},
     'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)': {'qualifiedName': 'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)',
      'layer': 'Service Layer'},
     'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)': {'qualifiedName': 'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)',
      'layer': 'Data Source Layer'},
     'de.schildbach.wallet.Configuration.disarmBackupReminder()': {'qualifiedName': 'de.schildba

In [20]:
def count_layer_occurrences(input_dict):
    layer_count = {}

    for _, details in input_dict.items():
        layer = details.get("layer")
        if layer:
            if layer in layer_count:
                layer_count[layer] += 1
            else:
                layer_count[layer] = 1

    return layer_count

In [21]:
for pkg_id in results:
	for cls_id in results[pkg_id]['classes']:
		results[pkg_id]['classes'][cls_id]['layers'] = count_layer_occurrences(results[pkg_id]['classes'][cls_id]['methods'])

results

{'de.schildbach.wallet': {'qualifiedName': 'de.schildbach.wallet',
  'classes': {'de.schildbach.wallet.Configuration': {'qualifiedName': 'de.schildbach.wallet.Configuration',
    'methods': {'de.schildbach.wallet.Configuration.armBackupReminder()': {'qualifiedName': 'de.schildbach.wallet.Configuration.armBackupReminder()',
      'layer': 'Data Source Layer'},
     'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)': {'qualifiedName': 'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)',
      'layer': 'Service Layer'},
     'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)': {'qualifiedName': 'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)',
      'layer': 'Data Source Layer'},
     'de.schildbach.wallet.Configuration.disarmBackupReminder()': {'qualifiedName': 'de.schildba

In [22]:
def sum_layer_counts(input_dicts):
    layer_count = {}

    for _, details in input_dicts.items():
        layers = details.get("layers", {})
        for layer, count in layers.items():
            if layer in layer_count:
                layer_count[layer] += count
            else:
                layer_count[layer] = count

    return layer_count

In [23]:
for pkg_id in results:
	results[pkg_id]['layers'] = sum_layer_counts(results[pkg_id]['classes'])

results

{'de.schildbach.wallet': {'qualifiedName': 'de.schildbach.wallet',
  'classes': {'de.schildbach.wallet.Configuration': {'qualifiedName': 'de.schildbach.wallet.Configuration',
    'methods': {'de.schildbach.wallet.Configuration.armBackupReminder()': {'qualifiedName': 'de.schildbach.wallet.Configuration.armBackupReminder()',
      'layer': 'Data Source Layer'},
     'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)': {'qualifiedName': 'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)',
      'layer': 'Service Layer'},
     'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)': {'qualifiedName': 'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)',
      'layer': 'Data Source Layer'},
     'de.schildbach.wallet.Configuration.disarmBackupReminder()': {'qualifiedName': 'de.schildba

In [24]:
write_to_json_file(results, f"../../test/2-architecture-layering-output/{project_name}-layers-recap-{model}-{timestr}.json")

In [25]:
rows = []

for pkg_id in results:
	for cls_id in [c for c in results[pkg_id]['classes']]:
		for met_id in [m for m in results[pkg_id]['classes'][cls_id]['methods']]:
			rows.append((pkg_id,cls_id,met_id,results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer']))

rows

[('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'de.schildbach.wallet.Configuration.armBackupReminder()',
  'Data Source Layer'),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'de.schildbach.wallet.Configuration.changeLogVersionCodeCrossed(int,int)',
  'Service Layer'),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'de.schildbach.wallet.Configuration.de.schildbach.wallet.Configuration(android.content.SharedPreferences,android.content.res.Resources)',
  'Data Source Layer'),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'de.schildbach.wallet.Configuration.disarmBackupReminder()',
  'Service Layer'),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'de.schildbach.wallet.Configuration.getBestChainHeightEver()',
  'Data Source Layer'),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'de.schildbach.wallet.Configuration.getBlockExplorer()',
  'Data Source Layer'),
 ('d

In [26]:
header = ("package", "class", "method", "layer")

In [27]:
import csv

with open(f"../../test/2-architecture-layering-output/{project_name}-layers1-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [28]:
rows = []

for pkg_id in results:
	for cls_id in [c for c in results[pkg_id]['classes']]:
		for layer in results[pkg_id]['classes'][cls_id]['layers']:
			rows.append((pkg_id,cls_id,layer,results[pkg_id]['classes'][cls_id]['layers'][layer]))

rows

[('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Data Source Layer',
  20),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Service Layer',
  6),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Domain Layer',
  7),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Cross-Cutting',
  2),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Constants',
  'Cross-Cutting',
  1),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Constants$Files',
  'Domain Layer',
  1),
 ('de.schildbach.wallet', 'de.schildbach.wallet.Logging', 'Cross-Cutting', 1),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Logging',
  'Data Source Layer',
  1),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.WalletApplication',
  'Cross-Cutting',
  6),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.WalletApplication',
  'Data Source Layer',
  4),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.WalletApplication',
  'Service Layer',
  

In [29]:
header = ("package", "class", "layer", "count")

In [30]:
import csv

with open(f"../../test/2-architecture-layering-output/{project_name}-layers2-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [31]:
rows = []

for pkg_id in results:
	for cls_id in [c for c in results[pkg_id]['classes']]:
		for layer in results[pkg_id]['classes'][cls_id]['layers']:
			rows.append((pkg_id,cls_id,layer,results[pkg_id]['classes'][cls_id]['layers'][layer]/sum(results[pkg_id]['classes'][cls_id]['layers'].values())))

rows

[('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Data Source Layer',
  0.5714285714285714),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Service Layer',
  0.17142857142857143),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Domain Layer',
  0.2),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Configuration',
  'Cross-Cutting',
  0.05714285714285714),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Constants',
  'Cross-Cutting',
  1.0),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Constants$Files',
  'Domain Layer',
  1.0),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Logging',
  'Cross-Cutting',
  0.5),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.Logging',
  'Data Source Layer',
  0.5),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.WalletApplication',
  'Cross-Cutting',
  0.4),
 ('de.schildbach.wallet',
  'de.schildbach.wallet.WalletApplication',
  'Data Source Layer',
  0.26666666666666666),
 ('

In [32]:
import csv

with open(f"../../test/2-architecture-layering-output/{project_name}-layers3-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [33]:
for pkg_id,package in results.items():
	for cls_id,clasz in package['classes'].items():
		for met_id,method in clasz['methods'].items():
			nodes[met_id]['properties']['layer'] = method['layer']

In [34]:
nodes

{'de.schildbach.wallet.ui.WalletBalanceFragment.onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater).inflater': {'id': 'de.schildbach.wallet.ui.WalletBalanceFragment.onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater).inflater',
  'properties': {'simpleName': 'inflater',
   'qualifiedName': 'de.schildbach.wallet.ui.WalletBalanceFragment.onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater).inflater',
   'kind': 'parameter',
   'metaSrc': 'source code'},
  'labels': ['Variable']},
 'de.schildbach.wallet.ui.TransactionsAdapter.de.schildbach.wallet.ui.TransactionsAdapter(android.content.Context,int,de.schildbach.wallet.ui.TransactionsAdapter$OnClickListener).context': {'id': 'de.schildbach.wallet.ui.TransactionsAdapter.de.schildbach.wallet.ui.TransactionsAdapter(android.content.Context,int,de.schildbach.wallet.ui.TransactionsAdapter$OnClickListener).context',
  'properties': {'simpleName': 'context',
   'qualifiedName': 'de.schildbach.wallet.ui.Transact

In [35]:
graph['elements']['nodes'] = [{'data':node_data} for node_data in nodes.values()]

In [37]:
write_to_json_file(graph,f'../../test/2-architecture-layering-output/{project_name}-with-layers-{timestr}.json')